In [ ]:
import numpy as np
import tensorflow as tf


def complex_flatten (real, imag):
    real = tf.keras.layers.Flatten()(real)
    imag = tf.keras.layers.Flatten()(imag)
    return real, imag


def CReLU (real, imag):
    real = tf.keras.layers.ReLU()(real)
    imag = tf.keras.layers.ReLU()(imag)
    return real, imag

def zReLU_new(real, imag):
    # ReLU applied separately
    real_relu = tf.keras.layers.ReLU()(real)
    imag_relu = tf.keras.layers.ReLU()(imag)

    # Create binary mask where both real and imag > 0
    real_flag = tf.cast(real_relu > 0, tf.float32)
    imag_flag = tf.cast(imag_relu > 0, tf.float32)
    mask = real_flag * imag_flag  # only keep values where both > 0

    # Apply mask
    real_out = real_relu * mask
    imag_out = imag_relu * mask
    return real_out, imag_out

def zReLU (real, imag):
    real = tf.keras.layers.ReLU()(real)
    imag = tf.keras.layers.ReLU()(imag)
    
    real_flag = tf.cast(tf.cast(real, tf.bool), tf.float32)
    imag_flag = tf.cast(tf.cast(imag, tf.bool), tf.float32)
    
    flag = real_flag * imag_flag

    real = tf.math.multiply(real, flag)
    imag = tf.math.multiply(imag, flag)
    return real, imag

import tensorflow as tf
import numpy as np

class ModReLU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ModReLU, self).__init__(**kwargs)
    
    def build(self, input_shape):
        # input_shape: tuple of (real_shape, imag_shape)
        channel_dim = input_shape[0][-1]  # assuming format [B, H, W, C]
        self.bias = self.add_weight(
            shape=(channel_dim,),
            initializer="zeros",
            trainable=True,
            name="modrelu_bias"
        )

    def call(self, inputs):
        real, imag = inputs

        z = tf.complex(real, imag)
        norm = tf.abs(z) + 1e-8  # avoid division by zero
        relu_out = tf.nn.relu(norm + self.bias)

        scale = relu_out / norm
        real_out = scale * real
        imag_out = scale * imag
        return real_out, imag_out

def modReLU (real, imag):
    norm = tf.abs(tf.complex(real, imag))
    bias = tf.Variable(np.zeros([norm.get_shape()[-1]]), trainable = True, dtype=tf.float32)
    relu = tf.nn.relu(norm + bias)
    
    real = tf.math.multiply(relu / norm + (1e+5), real)
    imag = tf.math.multiply(relu / norm + (1e+5), imag)
    return real, imag


def CLeaky_ReLU(real, imag):
    real = tf.nn.leaky_relu(real,alpha=0.1)
    imag = tf.nn.leaky_relu(imag,alpha=0.1)
    return real, imag

def CLeaky_ReLU_new(real, imag, alpha=0.1, name=None):
    real = tf.nn.leaky_relu(real, alpha=alpha, name=None if name is None else name + '_real')
    imag = tf.nn.leaky_relu(imag, alpha=alpha, name=None if name is None else name + '_imag')
    return real, imag

def complex_tanh (real, imag):
    real = tf.nn.tanh(real)
    imag = tf.nn.tanh(imag)
    return real, imag


def complex_softmax (real, imag):
    magnitude = tf.abs(tf.complex(real, imag))
    magnitude = tf.keras.layers.Softmax()(magnitude)
    return magnitude
    




In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class HRelu(Layer):
    def __init__(self, **kwargs):
        super(HRelu, self).__init__(**kwargs)

    def call(self, inputs):
        real, imag = inputs  # Expecting a tuple (real, imag)
        mask = tf.cast(imag >= 0, dtype=real.dtype)  # 1 if imag >= 0, else 0
        return real * mask, imag * mask  # Zeroing out both real and imag if imag < 0
